In [28]:
import time
import psutil
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.model_selection import cross_val_score

def extract_features(password: str):
    features = {
        'length': len(password),
        'uppercase': sum(1 for char in password if char.isupper()),
        'lowercase': sum(1 for char in password if char.islower()),
        'digits': sum(1 for char in password if char.isdigit()),
        'special_chars': sum(1 for char in password if not char.isalnum())
    }
    return features

def evaluate_model_accuracy(model, X_test, y_test):
    """วัดความแม่นยำของโมเดลด้วยหลายๆ metrics"""
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1
    }

def format_evaluation_results(results_df):
    """Format model evaluation results for better display"""
    results_df.columns = pd.MultiIndex.from_tuples([
        ('Model Info', 'Model'),
        ('Training', 'Training Time (s)'),
        ('Training', 'Training Memory (MB)'),
        ('Predictions', 'Predictions'), # เปลี่ยนจาก Total เป็น Predictions
        ('Performance', 'Prediction Time (s)'),
        ('Performance', 'Time/Record (ms)'),
        ('Performance', 'Prediction Memory (MB)'),
        ('Metrics', 'Time StdDev'),
        ('Metrics', 'Memory StdDev'),
        ('Model Accuracy', 'Accuracy'),
        ('Model Accuracy', 'Precision'),
        ('Model Accuracy', 'Recall'),
        ('Model Accuracy', 'F1-score'),
        ('Cross Validation', 'Mean CV Score'),
        ('Cross Validation', 'CV Score Std'),
        ('Model Complexity', 'Number of Parameters'),
        ('ROC Analysis', 'ROC AUC Scores')
    ])

    # ปรับการ format ให้ตรงกับคอลัมน์ใหม่ด้วย
    styled_df = results_df.style.format({
        ('Training', 'Training Time (s)'): '{:.3f}',
        ('Training', 'Training Memory (MB)'): '{:.2f}',
        ('Predictions', 'Predictions'): '{:,.0f}',
        ('Performance', 'Prediction Time (s)'): '{:.5f}',
        ('Performance', 'Time/Record (ms)'): '{:.6f}',
        ('Performance', 'Prediction Memory (MB)'): '{:.2f}',
        ('Metrics', 'Time StdDev'): '{:.6f}',
        ('Metrics', 'Memory StdDev'): '{:.6f}',
        ('Model Accuracy', 'Accuracy'): '{:.2%}',
        ('Model Accuracy', 'Precision'): '{:.2%}',
        ('Model Accuracy', 'Recall'): '{:.2%}',
        ('Model Accuracy', 'F1-score'): '{:.2%}',
        ('Cross Validation', 'Mean CV Score'): '{:.4f}',
        ('Cross Validation', 'CV Score Std'): '{:.4f}',
        ('Model Complexity', 'Number of Parameters'): '{:,.0f}',
    })

    styled_df = styled_df.set_properties(**{
        'text-align': 'center',
        'padding': '10px',
        'border': '1px solid #ddd'
    })

    styled_df = styled_df.set_table_styles([
        {'selector': 'thead tr:nth-child(1)',
         'props': [('background-color', '#4a5568'),
                  ('color', 'white'),
                  ('font-weight', 'bold'),
                  ('text-align', 'center'),
                  ('padding', '8px')]},
        {'selector': 'thead tr:nth-child(2)',
         'props': [('background-color', '#718096'),
                  ('color', 'white'),
                  ('font-weight', 'normal'),
                  ('text-align', 'center'),
                  ('padding', '8px')]},
        {'selector': 'td, th',
         'props': [('border', '1px solid #ddd'),
                  ('padding', '8px')]},
        {'selector': 'tbody tr:hover',
         'props': [('background-color', '#f8f9fa')]}
    ])

    styled_df = styled_df.background_gradient(
        subset=pd.IndexSlice[:, [('Model Accuracy', 'Accuracy'),
                                ('Model Accuracy', 'Precision'),
                                ('Model Accuracy', 'Recall'),
                                ('Model Accuracy', 'F1-score')]],
        cmap='RdYlGn',
        vmin=0,
        vmax=1
    )

    return styled_df

def measure_training_performance(model, X_train, y_train):
    """วัดประสิทธิภาพการเทรนโมเดล"""
    gc.collect()
    process = psutil.Process()
    memory_before = process.memory_info().rss / 1024 / 1024 # Convert to MB

    train_start = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - train_start

    memory_after = process.memory_info().rss / 1024 / 1024
    memory_used = memory_after - memory_before

    return {
        'Training Time (s)': train_time,
        'Training Memory (MB)': memory_used
    }

def measure_prediction_performance(models, X_test, n_trials=3):
    """วัดประสิทธิภาพการทำนายของโมเดล"""
    results = []

    for name, model in models:
        prediction_times = []
        memory_usages = []

        for _ in range(n_trials):
            gc.collect()
            time.sleep(0.1)

            process = psutil.Process()
            memory_before = process.memory_info().rss / 1024 / 1024

            predict_start = time.time()
            y_pred = model.predict(X_test)
            predict_time = time.time() - predict_start

            gc.collect()
            time.sleep(0.1)
            memory_after = process.memory_info().rss / 1024 / 1024
            memory_usage = max(0.001, memory_after - memory_before)

            prediction_times.append(predict_time)
            memory_usages.append(memory_usage)

        avg_predict_time = np.mean(prediction_times)
        avg_memory_usage = np.mean(memory_usages)

        results.append({
            'Model': name,
            'Predictions': len(X_test),
            'Prediction Time (s)': avg_predict_time,
            'Time/Record (ms)': (avg_predict_time / len(X_test)) * 1000,
            'Prediction Memory (MB)': avg_memory_usage,
            'Time StdDev': np.std(prediction_times),
            'Memory StdDev': np.std(memory_usages)
        })

    return pd.DataFrame(results)

def cross_validate_model(model, X, y, cv=5):
    """Cross-validation เพื่อประเมินความเสถียรของโมเดล"""
    cv_scores = cross_val_score(model, X, y, cv=cv)
    return {
        'Mean CV Score': cv_scores.mean(),
        'CV Score Std': cv_scores.std()
    }

def plot_roc_curves(model, X_test, y_test):
    """สร้าง ROC curves และคำนวณ AUC สำหรับแต่ละคลาส"""
    y_probs = model.predict_proba(X_test)

    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(len(np.unique(y_test))):
        fpr[i], tpr[i], _ = roc_curve(y_test == i, y_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    return {
        'FPR': fpr,
        'TPR': tpr,
        'ROC AUC': roc_auc
    }

def measure_model_complexity(model):
    """วัดความซับซ้อนของโมเดล"""
    if hasattr(model, 'coef_'):
        n_parameters = np.sum([np.prod(p.shape) for p in model.coef_])
        return {
            'Number of Parameters': n_parameters
        }
    return {
        'Number of Parameters': 'Not Available'
    }

def comprehensive_model_evaluation(model_name, model, X_train, X_test, y_train, y_test):
    """ประเมินโมเดลแบบครอบคลุมทุกด้าน"""
    results = {'Model': model_name}

    results.update(measure_training_performance(model, X_train, y_train))

    pred_results = measure_prediction_performance([(model_name, model)], X_test)
    results.update(pred_results.iloc[0].to_dict())

    results.update(evaluate_model_accuracy(model, X_test, y_test))

    results.update(cross_validate_model(model, X_train, y_train))

    results.update(measure_model_complexity(model))

    roc_results = plot_roc_curves(model, X_test, y_test)
    results['ROC AUC Scores'] = roc_results['ROC AUC']

    return pd.DataFrame([results])

def evaluate_password_strength_model(model_name, model, data_file):
    """Evaluate and display formatted results for password strength model"""
    print(f"\n{'='*20} Evaluating {model_name} {'='*20}\n")

    data = pd.read_csv(data_file, on_bad_lines='skip')
    data = data.dropna(subset=['password'])

    features = []
    for pwd in data['password']:
        features.append(extract_features(pwd))
    X = pd.DataFrame(features)
    y = data['strength']

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    results = comprehensive_model_evaluation(model_name, model, X_train, X_test, y_train, y_test)

    # Format and display results
    styled_results = format_evaluation_results(results)
    display(styled_results)

    return results


import joblib

lr_model = joblib.load('lr_model.pkl')

models = [
    ('Logistic Regression', lr_model),
]

for model_name, model in models:
    results = evaluate_password_strength_model(model_name, model, 'data.csv')


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(



==================== Evaluating Logistic Regression ====================



In [36]:
import pandas as pd
import random
import string
import re

def extract_features(password: str):
    """Extract password features for classification"""
    features = {
        'length': len(password),
        'uppercase': sum(1 for char in password if char.isupper()),
        'lowercase': sum(1 for char in password if char.islower()),
        'digits': sum(1 for char in password if char.isdigit()),
        'special_chars': sum(1 for char in password if not char.isalnum())
    }
    return features

def evaluate_password_strength(password):
    """
    ประเมินความแข็งแรงของรหัสผ่านตาม features
    Returns: 0 (อ่อน), 1 (ปานกลาง), 2 (แข็งแรง)
    """
    features = extract_features(password)

    # เกณฑ์ระดับ 0 (อ่อน)
    # - ความยาว <= 8
    # - มีแค่ตัวพิมพ์เล็กล้วน หรือ ตัวเลขล้วน
    if features['length'] <= 8:
        if (features['digits'] == features['length'] and features['digits'] > 0) or \
           (features['lowercase'] == features['length'] and features['lowercase'] > 0):
            return 0

    # เกณฑ์ระดับ 2 (แข็งแรง)
    # - ความยาว >= 12
    # - มีตัวพิมพ์ใหญ่อย่างน้อย 2 ตัว
    # - มีตัวพิมพ์เล็กอย่างน้อย 2 ตัว
    # - มีตัวเลขอย่างน้อย 2 ตัว
    # - มีอักขระพิเศษอย่างน้อย 2 ตัว
    if features['length'] >= 12 and \
       features['uppercase'] >= 2 and \
       features['lowercase'] >= 2 and \
       features['digits'] >= 2 and \
       features['special_chars'] >= 2:
        return 2

    # เกณฑ์ระดับ 1 (ปานกลาง)
    # - ความยาว 8-12 ตัว
    # - มีตัวอักษร (พิมพ์เล็กหรือพิมพ์ใหญ่) ผสมกับตัวเลข
    # - ไม่มีอักขระพิเศษ
    if 8 <= features['length'] <= 12 and \
       features['special_chars'] == 0 and \
       features['digits'] >= 1 and \
       (features['lowercase'] >= 1 or features['uppercase'] >= 1):
        return 1

    return -1

def generate_password_dataset(n_samples=100000):
    """สร้างชุดข้อมูลรหัสผ่านตามเกณฑ์ที่กำหนด"""
    passwords = []
    strengths = []

    def generate_weak_password():
        """สร้างรหัสผ่านระดับ 0 (อ่อน)"""
        choices = [
            # ตัวเลขล้วน 6-8 หลัก
            lambda: ''.join(random.choices(string.digits, k=random.randint(6, 8))),
            # ตัวอักษรพิมพ์เล็กล้วน 6-8 ตัว
            lambda: ''.join(random.choices(string.ascii_lowercase, k=random.randint(6, 8)))
        ]
        return random.choice(choices)()

    def generate_medium_password():
        """สร้างรหัสผ่านระดับ 1 (ปานกลาง)"""
        length = random.randint(8, 12)
        n_digits = random.randint(2, 4)
        n_letters = length - n_digits

        # สร้างด้วยตัวพิมพ์เล็กและตัวเลข (ไม่มีตัวพิมพ์ใหญ่หรืออักขระพิเศษ)
        password = ''.join(random.choices(string.ascii_lowercase, k=n_letters))
        password += ''.join(random.choices(string.digits, k=n_digits))

        password_list = list(password)
        random.shuffle(password_list)
        return ''.join(password_list)

    def generate_strong_password():
        """สร้างรหัสผ่านระดับ 2 (แข็งแรง)"""
        # กำหนดจำนวนอย่างน้อยของแต่ละประเภท
        n_lower = random.randint(2, 4)
        n_upper = random.randint(2, 4)
        n_digits = random.randint(2, 4)
        n_special = random.randint(2, 4)

        # คำนวณความยาวทั้งหมด (12-16 ตัว)
        min_length = max(12, n_lower + n_upper + n_digits + n_special)
        length = random.randint(min_length, 16)

        # สร้างรหัสผ่าน
        password = ''.join(random.choices(string.ascii_lowercase, k=n_lower))
        password += ''.join(random.choices(string.ascii_uppercase, k=n_upper))
        password += ''.join(random.choices(string.digits, k=n_digits))
        password += ''.join(random.choices('!@#$%^&*()_+-=[]{}|;:,.<>?', k=n_special))

        # เพิ่มตัวอักษรที่เหลือ
        remaining = length - len(password)
        if remaining > 0:
            password += ''.join(random.choices(
                string.ascii_letters + string.digits + '!@#$%^&*',
                k=remaining
            ))

        # สลับตำแหน่ง
        password_list = list(password)
        random.shuffle(password_list)
        return ''.join(password_list)

    # สร้างตามสัดส่วนที่กำหนด
    distributions = {
        0: int(n_samples * 0.134),  # 13.4% รหัสผ่านอ่อน
        1: int(n_samples * 0.742),  # 74.2% รหัสผ่านปานกลาง
        2: int(n_samples * 0.124)   # 12.4% รหัสผ่านแข็งแรง
    }

    for strength, count in distributions.items():
        generated = 0
        attempts = 0
        while generated < count and attempts < count * 2:
            attempts += 1

            if strength == 0:
                password = generate_weak_password()
            elif strength == 1:
                password = generate_medium_password()
            else:
                password = generate_strong_password()

            # ตรวจสอบว่าตรงตามเกณฑ์
            evaluated_strength = evaluate_password_strength(password)
            if evaluated_strength == strength:
                passwords.append(password)
                strengths.append(strength)
                generated += 1

    # สร้าง DataFrame
    df = pd.DataFrame({
        'password': passwords,
        'strength': strengths
    })

    # สลับข้อมูล
    df = df.sample(frac=1).reset_index(drop=True)

    return df

# ทดสอบการสร้างชุดข้อมูล
if __name__ == "__main__":
    test_data = generate_password_dataset(n_samples=100000)
    print("\nTest Dataset Summary:")
    print(f"Total samples: {len(test_data)}")
    print("\nDistribution of password strengths:")
    print(test_data['strength'].value_counts().sort_index())

    # แสดงตัวอย่างและ features
    print("\nExample passwords and their features:")
    for strength in [0, 1, 2]:
        print(f"\nStrength {strength}:")
        sample_passwords = test_data[test_data['strength'] == strength]['password'].head(3)
        for pwd in sample_passwords:
            print(f"Password: {pwd}")
            print(f"Features: {extract_features(pwd)}")
    test_data = generate_password_dataset(n_samples=100000)

    # Export เป็น CSV
    output_file = 'generated_passwords.csv'
    test_data.to_csv(output_file, index=False)
    print(f"\nExported dataset to: {output_file}")



Test Dataset Summary:
Total samples: 100000

Distribution of password strengths:
strength
0    13400
1    74200
2    12400
Name: count, dtype: int64

Example passwords and their features:

Strength 0:
Password: 2592529
Features: {'length': 7, 'uppercase': 0, 'lowercase': 0, 'digits': 7, 'special_chars': 0}
Password: 26596031
Features: {'length': 8, 'uppercase': 0, 'lowercase': 0, 'digits': 8, 'special_chars': 0}
Password: hopottlh
Features: {'length': 8, 'uppercase': 0, 'lowercase': 8, 'digits': 0, 'special_chars': 0}

Strength 1:
Password: okcacf2y3oo
Features: {'length': 11, 'uppercase': 0, 'lowercase': 9, 'digits': 2, 'special_chars': 0}
Password: l10jpryyty
Features: {'length': 10, 'uppercase': 0, 'lowercase': 8, 'digits': 2, 'special_chars': 0}
Password: p28tdsiuq6
Features: {'length': 10, 'uppercase': 0, 'lowercase': 7, 'digits': 3, 'special_chars': 0}

Strength 2:
Password: ++9u8bn{UZ9}V8
Features: {'length': 14, 'uppercase': 3, 'lowercase': 3, 'digits': 4, 'special_chars': 4}


In [37]:
def test_on_unseen_data(model_name, model, test_data_path='generated_passwords.csv'):
    """ทดสอบโมเดลกับข้อมูลที่ไม่เคยเห็น"""
    print(f"\n{'='*20} Testing on Unseen Data: {model_name} {'='*20}\n")

    # โหลดข้อมูลทดสอบ
    test_data = pd.read_csv(test_data_path)

    # สร้าง features
    features = []
    for pwd in test_data['password']:
        features.append(extract_features(pwd))
    X_test = pd.DataFrame(features)
    y_test = test_data['strength']

    # ทำนายและประเมินผล
    y_pred = model.predict(X_test)

    results = {
        'Model': model_name,
        'Total Test Samples': len(test_data),
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1-score': f1_score(y_test, y_pred, average='weighted')
    }

    # แสดงผลลัพธ์
    print("\nModel Performance on Unseen Data:")
    for metric, value in results.items():
        if isinstance(value, (int, float)):
            if metric == 'Total Test Samples':
                print(f"{metric}: {value:,}")
            else:
                print(f"{metric}: {value:.4f}")

    # แสดงตัวอย่างการทำนายผิด
    print("\nExample Misclassifications:")
    misclassified = test_data[y_pred != y_test].sample(min(5, len(test_data[y_pred != y_test])))
    for _, row in misclassified.iterrows():
        print(f"\nPassword: {row['password']}")
        print(f"True Strength: {row['strength']}")
        print(f"Predicted Strength: {y_pred[row.name]}")

    return results

# ทดสอบโมเดล
import joblib
lr_model = joblib.load('lr_model.pkl')
unseen_results = test_on_unseen_data('Logistic Regression', lr_model)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(



==================== Testing on Unseen Data: Logistic Regression ====================


Model Performance on Unseen Data:
Total Test Samples: 100,000
Accuracy: 0.9205
Precision: 0.9282
Recall: 0.9205
F1-score: 0.9152

Example Misclassifications:

Password: 65062497
True Strength: 0
Predicted Strength: 1

Password: q0FQ!Wa=oz74
True Strength: 2
Predicted Strength: 1

Password: 14785545
True Strength: 0
Predicted Strength: 1

Password: uD%R5^8U@J2z
True Strength: 2
Predicted Strength: 1

Password: =l3w?<Et50O[
True Strength: 2
Predicted Strength: 1
